In [1]:
# This section of the code is for all of my imports
import os
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import pandas as pd
from datetime import datetime

# This allows me to access my flash drive that im working off of
os.chdir('/Volumes/Flash Drive')

In [ ]:
# This section loads all CSV files from the directory
import glob

Path = '/Volumes/Flash Drive/Saturns rings Research/Data From Center of Ringlets CSV files/'

# Get all CSV files in the directory
csv_files = glob.glob(os.path.join(Path, '*.csv'))

print(f"Found {len(csv_files)} CSV files:")
for csv_file in csv_files:
    print(f"  - {os.path.basename(csv_file)}")

In [3]:
# This Section is the formulas to convert from UTC to ET time, im working off of ET time

# This formula gets the UTC and ET offset due to leap seconds, number is in seconds
def utc_to_et_offset(year):
    """
    Get the offset between UTC and ET (TDB) in seconds for 2008.
    
    For 2008:
    - Leap seconds accumulated by 2008: 33 seconds
    - TT-TAI offset: 32.184 seconds
    - ET ≈ TDB ≈ TT for most purposes
    - So ET - UTC ≈ 33 + 32.184 = 65.184 seconds
    """
    
    ls = 33  # Leap Seconds since 2008

    # TT - TAI offset is always 32.184 seconds
    tt_tai_offset = 32.184
    
    # ET ≈ TDB ≈ TT = UTC + leap_seconds + 32.184
    et_utc_offset = ls + tt_tai_offset
    
    return et_utc_offset # Seconds

# This section of the code converts the UTC Julian Date to ET Julian Date adding in the leap seconds offset
def convert_paper_time_to_et(jd_utc):
    """
    Convert the paper's UTC-based Julian Date to ET-based Julian Date.
    
    Parameters:
    -----------
    jd_utc : float
        Julian Date in UTC (as used in the paper)
    
    Returns:
    --------
    jd_et : float
        Julian Date in Ephemeris Time
    """
    
    # Get offset for 2008
    et_utc_offset_2008 = utc_to_et_offset(2008)
    
    # Convert to ET
    initial_time = jd_utc + (et_utc_offset_2008 / 86400.0)
    
    return initial_time # Seconds


In [4]:
# This section defines the True Anamoly formula and the radius formula

def calculate_radius_true_anomaly(a, e, true_anomaly):
    """
    Calculate the radius at a given true anomaly for a Keplerian ellipse.
    
    This implements equation (2) from the paper:
    r(λ,t) = a(1 - e²) / (1 + e·cos(f))
    f = λ - ϖ = λ - ϖ₀ - ϖ̇(t - t₀)
    
    Parameters:
    -----------
    a : float
        Semi-major axis (km)
    e : float
        Eccentricity (dimensionless, between 0 and 1)
    true_anomaly : float or array
        True anomaly f = λ - ϖ = λ - ϖ₀ - ϖ̇(t - t₀)
        where λ is the inertial longitude and ϖ is the longitude of periapse
    
    Returns:
    --------
    r : float or array
        Radius at the given true anomaly (km)
    """

        # Convert to radians
    #true_anomaly = true_anomaly * np.pi / 180
    
    # Calculate the numerator: a(1 - e²)
    numerator = a * (1 - e**2)
    
    # Calculate the denominator: 1 + e·cos(f)
    denominator = 1 + e * np.cos(true_anomaly)
    
    # Calculate radius
    r = numerator / denominator
    
    return r

def calculate_true_anomaly(longitude #Inertial longitude (LON value)
                           ,varpi_0 # Longitude periapse (Fixed)
                           ,varpi_dot #Rrecession rate (Fixed)
                           ,time # Time from data
                           ,initial_time): # Time from paper (fixed)
    """
    Calculate the true anomaly from orbital parameters.
    
    From the paper: f = λ - ϖ = λ - ϖ₀ - ϖ̇(t - t₀)
    
    Parameters:
    -----------
    longitude : float or array
        Inertial longitude λ (degrees)
    varpi_0 (Longitude periapse) : float
        Longitude of periapse at epoch ϖ₀ (degrees)
    varpi_dot (precession rate) : float, optional
        Apsidal precession rate ϖ̇ (degrees/day)
    time : float, optional
        Current time (days)
    initial_time : float, optional
        Epoch time t₀ (days)
    
    Returns:
    --------
    true_anomaly : float or array
        True anomaly f (degrees)
    """
    
    # True anomaly is the angle from periapse
    true_anomaly = longitude - varpi_0 - varpi_dot * (time - initial_time)

    # Wrap to 0-360 degrees
    true_anomaly = true_anomaly % 360
    
    return true_anomaly

def plot_r_vs_true_anomaly(true_anomaly, radii, title=f"\Radius vs True Anomaly Analysis of Titan Ringlet \n File: {CSV}"):
    """
    Plot radius vs true anomaly.
    
    Parameters:
    -----------
    true_anomaly : array
        True anomaly values in degrees
    radii : array
        Radius values in km
    title : str
        Plot title
    """
    plt.figure(figsize=(10, 6))
    plt.plot(true_anomaly, radii, 'b-', linewidth=2)
    plt.xlabel('True Anomaly, f (degrees)')
    plt.ylabel('Radius (km)')
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.show()

In [ ]:
# This section defines static parameters and initializes data storage

# J2000 epoch = JD 2451545.0
j2000_jd = 2451545.0

# Paper's epoch
paper_epoch_utc = 2454467.0  # This is in UTC

# Paper's epoch conversion to ET (seconds)
paper_epoch_jd_et = convert_paper_time_to_et(paper_epoch_utc)

# Static orbital parameters (these don't change between occultations)
ae = 17.39    # km
varpi_0 = 270.54  # degrees
varpi_dot = 22.57503  # ϖ̇ in degrees/day

# Initialize lists to store all data points from all occultations
all_true_anomaly = []
all_radii = []
all_longitudes = []
all_times = []
all_csv_names = []

print("Fixed orbital parameters:")
print(f"  ae = {ae} km")
print(f"  varpi_0 = {varpi_0}°")
print(f"  varpi_dot = {varpi_dot}°/day")
print(f"  paper_epoch_jd_et = {paper_epoch_jd_et} JD")

In [ ]:
# This section loops through all CSV files and processes each one

print(f"\nProcessing {len(csv_files)} CSV files...\n")

for csv_file in csv_files:
    csv_name = os.path.basename(csv_file)
    print(f"Processing: {csv_name}")
    
    try:
        # Load data from CSV
        data = pd.read_csv(csv_file)
        
        # Get Titan ringlet data
        titan_data = data[data['Ringlet'] == 'Titan']
        
        if len(titan_data) == 0:
            print(f"  ⚠ No Titan ringlet data found in {csv_name}")
            continue
        
        # Extract radius value from CSV (mean of all radius values for this occultation)
        # Note: This assumes 'Radius' column exists. If not, will use default value.
        if 'Radius' in titan_data.columns:
            a = titan_data['Radius'].values.mean()  # Use mean radius
            print(f"  ✓ Using radius from CSV: a = {a:.2f} km")
        else:
            a = 77885.50  # Fallback to default if Radius column doesn't exist yet
            print(f"  ⚠ 'Radius' column not found, using default: a = {a:.2f} km")
        
        # Calculate eccentricity
        e = ae / a
        
        # Get longitude from data
        longitude = titan_data['LON'].values  # LON in degrees
        
        # Convert ET time from seconds to days
        titan_data_et_days = titan_data['ET'].values / 86400.0
        
        # Convert to Julian Days
        titan_data_jd_et = j2000_jd + titan_data_et_days
        
        # Calculate true anomaly for this data (comes out in degrees after wrapping)
        true_anomaly = calculate_true_anomaly(longitude, varpi_0, varpi_dot, 
                                             titan_data_jd_et, paper_epoch_jd_et)
        
        # Convert to radians before calculating radius
        true_anomaly_rad = true_anomaly * np.pi / 180
        
        # Calculate radii for this data
        radii = calculate_radius_true_anomaly(a, e, true_anomaly_rad)
        
        # Store all data points
        all_true_anomaly.extend(true_anomaly)
        all_radii.extend(radii)
        all_longitudes.extend(longitude)
        all_times.extend(titan_data_jd_et)
        all_csv_names.extend([csv_name] * len(true_anomaly))
        
        print(f"  ✓ Processed {len(true_anomaly)} data points")
        
    except Exception as ex:
        print(f"  ✗ Error processing {csv_name}: {ex}")
        continue

# Convert lists to numpy arrays for easier manipulation
all_true_anomaly = np.array(all_true_anomaly)
all_radii = np.array(all_radii)
all_longitudes = np.array(all_longitudes)
all_times = np.array(all_times)

print(f"\n{'='*60}")
print(f"TOTAL DATA POINTS COLLECTED: {len(all_true_anomaly)}")
print(f"{'='*60}")
print(f"True anomaly range: {np.min(all_true_anomaly):.2f}° to {np.max(all_true_anomaly):.2f}°")
print(f"Radii range: {np.min(all_radii):.2f} to {np.max(all_radii):.2f} km")
print(f"Time range: {np.min(all_times):.2f} to {np.max(all_times):.2f} JD")

In [ ]:
# This part creates the model curve using paper parameters

# Create model curve (sampling all true anomalies)
true_anomaly_deg_model = np.linspace(0, 360, 1000)
true_anomaly_rad_model = true_anomaly_deg_model * np.pi / 180

# Paper parameters for model
a_paper = 77867.13  # km
ae_paper = 17.39    # km 
e_paper = ae_paper / a_paper

model_radii = calculate_radius_true_anomaly(a_paper, e_paper, true_anomaly_rad_model)

print(f"Model parameters:")
print(f"  a_paper = {a_paper} km")
print(f"  e_paper = {e_paper:.6f}")
print(f"  Model radius range: {np.min(model_radii):.2f} to {np.max(model_radii):.2f} km")

In [ ]:
# This section plots the model curve with ALL data points from all occultations

plt.figure(figsize=(12, 8))

# Plot model curve
plt.plot(true_anomaly_deg_model, model_radii - a_paper, 'b-', linewidth=2, label='Model (Paper parameters)')

# Plot all data points from all occultations
plt.plot(all_true_anomaly, all_radii - a_paper, 'ro', markersize=8, label=f'Data ({len(all_true_anomaly)} points)', alpha=0.6)

# Formatting
plt.xlabel('True Anomaly, f (degrees)', fontsize=12)
plt.ylabel('r - a (km)', fontsize=12)
plt.title(f'Titan Ringlet - True Anomaly Analysis\nCombined data from {len(csv_files)} occultations', fontsize=14)
plt.xlim(0, 360)
plt.ylim(-50, 20)
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3, label='a (semi-major axis)')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

# Add text box with statistics
stats_text = f'Total points: {len(all_true_anomaly)}\n'
stats_text += f'Occultations: {len(csv_files)}\n'
stats_text += f'r-a range: {np.min(all_radii - a_paper):.2f} to {np.max(all_radii - a_paper):.2f} km'
plt.text(0.02, 0.98, stats_text, transform=plt.gca().transAxes, 
         fontsize=9, verticalalignment='top', 
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print(f"\nPlot generated with {len(all_true_anomaly)} data points from {len(csv_files)} CSV files")